# Initial look at the unifesp Kaggle data

Abdomen = 0
Ankle = 1
Cervical Spine = 2
Chest = 3
Clavicles = 4
Elbow = 5
Feet = 6
Finger = 7
Forearm = 8
Hand = 9
Hip = 10
Knee = 11
Lower Leg = 12
Lumbar Spine = 13
Others = 14
Pelvis = 15
Shoulder = 16
Sinus = 17
Skull = 18
Thigh = 19
Thoracic Spine = 20
Wrist = 21

In [2]:
#%%bash

#exec bash
!rm -rf data
!mkdir data
!apt install unzip

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



In [3]:
pip install -U timm==0.6.2dev0

Note: you may need to restart the kernel to use updated packages.


In [4]:
!cd data && kaggle competitions download -c unifesp-x-ray-body-part-classifier && unzip -q unifesp-x-ray-body-part-classifier

unifesp-x-ray-body-part-classifier.zip: Skipping, found more recently modified local copy (use --force to force download)


In [1]:
import timm

In [2]:
ls data/

archive.zip                             test_df.csv
comp_orig/                              train2_df.csv
image_png.png                           train_df.csv
images/                                 unifesp-x-ray-body-part-classifier.zip
sample_submission.csv


In [3]:
ls data/images/train | head

1.2.826.0.1.3680043.8.498.10025629581362719970278200333618114258-c.png
1.2.826.0.1.3680043.8.498.10036150326276641158002573300029848125-c.png
1.2.826.0.1.3680043.8.498.10038426859954986240523417641213777162-c.png
1.2.826.0.1.3680043.8.498.10050991192143676483577217922037244257-c.png
1.2.826.0.1.3680043.8.498.10053309524595490852815579052225336016-c.png
1.2.826.0.1.3680043.8.498.10053755320637729867508668285241208441-c.png
1.2.826.0.1.3680043.8.498.10062189329714053601496804394945741428-c.png
1.2.826.0.1.3680043.8.498.10065930002825553435161793347987832017-c.png
1.2.826.0.1.3680043.8.498.10072001800484199846734151421006166369-c.png
1.2.826.0.1.3680043.8.498.10077219503857952412254959562456136784-c.png


In [4]:
ls data/images/test | head

1.2.826.0.1.3680043.8.498.10001001190452685542754353271306894106-c.png
1.2.826.0.1.3680043.8.498.10022667601042710442884728915610557590-c.png
1.2.826.0.1.3680043.8.498.10024395388921105474336784272390605225-c.png
1.2.826.0.1.3680043.8.498.10026689165626095651568657562992631805-c.png
1.2.826.0.1.3680043.8.498.10035936364561920980924160549027696414-c.png
1.2.826.0.1.3680043.8.498.10048151016194111821946577128316853508-c.png
1.2.826.0.1.3680043.8.498.10050288866897098549171724525519515056-c.png
1.2.826.0.1.3680043.8.498.10058159876923426607443905471214229386-c.png
1.2.826.0.1.3680043.8.498.10080393723513249121106839017290899983-c.png
1.2.826.0.1.3680043.8.498.10111879479980116660072960146970217082-c.png


In [5]:
import pandas as pd

sample_sub = pd.read_csv('data/sample_submission.csv')
sample_sub.head()

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10001001190452685542...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,1.2.826.0.1.3680043.8.498.10022667601042710442...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2,1.2.826.0.1.3680043.8.498.10024395388921105474...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
3,1.2.826.0.1.3680043.8.498.10026689165626095651...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
4,1.2.826.0.1.3680043.8.498.10035936364561920980...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...


In [6]:
df = pd.read_csv('data/train2_df.csv')

Split the 2nd column on space, so the categories have their own columns.  Then just do single classification first.

In [7]:
#df.insert(2, "Target2","",True)
#df.insert(3, "Target3","",True)
#df.insert(4, "Target4","",True)
#df.insert(5, "Target5","",True)
#df[['Target1','Target2','Target3','Target4','Target5']] = df['Target'].str.split(' ', expand=True)

In [8]:
from fastai.vision.all import *
from fastcore.parallel import *

In [9]:
path = Path('data')
trn_path= path/'train'
tst_files = get_image_files(path/'test').sorted()

In [10]:
ImageDataLoaders.from_csv??

In [11]:
tta_res = []

In [12]:
def train(desc, arch, item, batch, accum=False):
    kwargs = {'bs':32} if accum else{}
    dls = ImageDataLoaders.from_csv(path, csv_fname='train2_df.csv', fn_col=-1, label_col=1, valid_pct=0.2, item_tfms=item, batch_tfms=batch, **kwargs)
    cbs = GradientAccumulation(2) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    learn.fine_tune(3, 0.01)
    tta_res.append(learn.tta(dl=dls.test_dl(tst_files)))
    learn.export(f'{arch}_{desc}')

In [13]:
timm.list_models("convnext_tiny*")

['convnext_tiny',
 'convnext_tiny_384_in22ft1k',
 'convnext_tiny_hnf',
 'convnext_tiny_hnfd',
 'convnext_tiny_in22ft1k',
 'convnext_tiny_in22k']

In [14]:
arch = ("convnext_tiny_in22k")

In [15]:
train('Convnext', arch, item=Resize(240, method='squish'), batch=aug_transforms(size=224), accum=False)

/Users/bsmi067/mambaforge/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/bsmi067/mambaforge/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,error_rate,time
0,1.886022,0.612007,0.138329,07:50


epoch,train_loss,valid_loss,error_rate,time
0,0.603974,0.440341,0.086455,12:11
1,0.534684,0.420477,0.100865,14:14
2,0.416039,0.348303,0.080692,12:18


IndexError: list index out of range

In [ ]:
save_pickle('tta_res_convnextM1.pkl', tta_res)

In [ ]:
tta_res=load_pickle('tta_res_convnextM16k.pkl')

In [ ]:
tta_prs = first(zip(*tta_res))

In [ ]:
t_tta = torch.stack(tta_prs)

In [ ]:
avg_pr = t_tta.mean(0)
idxs = avg_pr.argmax(dim=1)
idxs.shape

In [ ]:
# needs work to get the right sub format
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=Resize(224))
mapping = dict(enumerate(dls.vocab))
ss = pd.read_csv('data/sample_submission.csv')
results = pd.Series(idxs.numpy(), name='idxs').map(mapping)
ss.label = results
ss.to_csv('data/submissions/subm.csv', index=False)

In [ ]:
ss

In [ ]:
!kaggle competitions submit -c unifesp-x-ray-body-part-classifier -f data/submissions/subm.csv -m "M1 ConvNext"